In [1]:
import json
from time import strftime, strptime

In [2]:
dataDir = '../data/external/'

trumpSample = dataDir + 'trump/donaldtrump_1449187200.txt'
trumpFull = dataDir + 'trump/donaldtrump_1477699200.txt'

In [3]:
# Helper to read the ndljson's
def readNDLJson(f):
    with open(f) as data:
        jsonList = [json.loads(line) for line in data]
    return jsonList

In [4]:
# Helper function to take a string and return a python datetime obj
def tweetTime(created_at):
    # u'Fri Dec 04 00:00:32 +0000 2015'
    stamp = strptime(created_at, "%a %b %d %H:%M:%S +0000 %Y")
    
    return stamp

t1 = tweetTime('Fri Dec 04 00:00:32 +0000 2015')
t2 = tweetTime('Fri Dec 04 00:00:32 +0000 2016')
t1 > t2

False

In [5]:
# Function to take a tweet, and conditionally update the dictionairy
# It takes a json input, and first checks if it's a API rate limit call
# 

def updateDict(tweet):
    
    # occasionally we'll get an API call from twitter showing up, pass those
    try:
        if tweet['limit']:
            return
    except:
        pass
        
    try:
        # Sanity check
        if ((tweet['user']) and (tweet['user']['id_str'])):
        
            user = tweet['user']
            userID = tweet['user']['id_str']

            # grab timestamp, convert timestamp
            timestamp = tweetTime(tweet['created_at'])

            # dictionairy check
            if twitterDict.has_key(userID):
                # Compare if this tweet is newer than existing data
                if (timestamp > twitterDict[userID]['timestamp']):

                    # Update friends, followers/ing, statuses
                    twitterDict[userID]['following'] = user['friends_count']
                    twitterDict[userID]['followed'] = user['followers_count']
                    twitterDict[userID]['tweetsAll'] = user['statuses_count']

                    #and the timestamp!
                    twitterDict[userID]['timestamp'] = timestamp


                # either way, count the tweet
                twitterDict[userID]['tweetsDay'] = twitterDict[userID]['tweetsDay'] + 1

            # doesn't have key, instantiate!
            else:           
                twitterDict[userID] = {'tweetsDay': 1, 'userID': userID}

                # Update friends, followers/ing, statuses
                twitterDict[userID]['following'] = user['friends_count']
                twitterDict[userID]['followed'] = user['followers_count']
                twitterDict[userID]['tweetsAll'] = user['statuses_count']
                #and the timestamp!
                twitterDict[userID]['timestamp'] = timestamp
    except:
        #print (tweet)
        pass

In [6]:
twitterDict = {}
def readLine():
    global twitterDict
    corruptedLines = 0
    
    with open(trumpFull) as f:
        for line in f:
            try:
                updateDict(json.loads(line))
            except:
                corruptedLines += 1
    print (corruptedLines)
                
def readLines(x):
    #twitterDict = {}
    global twitterDict
    with open(trumpSample) as f:
        for chunk in f.readlines(x):
            for line in chunk:
                updateDict(json.loads(line))
    print (len(twitterDict.keys()))
    return twitterDict

In [7]:
readLine()

3


In [8]:
#%timeit -n 3 readLine()
#%timeit -n 1 test1 = readLines(1)
#%timeit -n 1 test10 = readLines(10)
#%timeit -n 1 test100 = readLines(100)
#%timeit -n 1 test1000 = readLines(1000)
#%timeit -n 1 test10000 = readLines(10000)
#%timeit -n 1 test100000 = readLines(1000000)

In [9]:
len(twitterDict.keys())

193608

In [10]:
import pandas as pd
aggTweets = pd.DataFrame.from_dict(twitterDict, orient='index')

In [11]:
aggTweets['timestamp'] = aggTweets['timestamp'].apply(lambda x: strftime("%Y-%m-%d %H:%M:%S", x))

In [12]:
aggTweets.to_csv('../data/processed/trumpAgg.csv', index=False)

### To Do

1. Compartmentalize the `twitterDict` into function
2. Create function writeoutCSV() to write out to `donald/hilary_1XXXXX-agg.csv`
3. Write code to take either a path/file and `readFile(f)` and `writeoutCSV(f-agg.csv)` 